# Examining item thumbnail size

## Setting Up the Notebook

In [1]:
import sys
prefix = '\"%s\"' %  sys.prefix
!conda install --yes --prefix {prefix} pillow

Solving environment: ...working... done

# All requested packages already installed.



## About Item Thumbnails

### Finding Missing and Invalid Images

In [2]:
import os
import io
import base64
import shutil
from getpass import getpass

In [3]:
import pandas as pd
from PIL import Image
from IPython.display import HTML
import ipywidgets as widgets
from arcgis.gis import GIS, Item, User

pd.set_option('display.max_colwidth', -1)

### Connect to the GIS

In [4]:
username = getpass()

········


In [5]:
password = getpass()

········


In [6]:
gis = GIS(username=username, password=password, verify_cert=False)

In [7]:
def get_images(username, gis, min_width=600, min_height=400, show_image=True):
    """
    Finds all images for a given user under a specific image size in pixels.
    
    ================   ===========================================
    **Inputs**         **Description**
    ----------------   -------------------------------------------
    username           Required string. The name of the user items 
                       to exmine.
    ----------------   -------------------------------------------
    gis                Required GIS. The connection to Portal/AGOL
    ----------------   -------------------------------------------
    min_height         Optional Integer. The height in pixels of 
                       the image. Any image below this height will 
                       be returned in the dataframe.
    ----------------   -------------------------------------------
    min_width          Optional Integer. The width of the image. 
                       Anything below this width will be returned.
    ----------------   -------------------------------------------
    show_image         Optional boolean. If True, the results will 
                       be returned as an HTML table. Else, they 
                       will be returned as a pandas DataFrame.
    ================   ===========================================
    
    returns: string or pd.DataFrame
    
    """
    results = []
    show_image_columns=['title', 'username','folder', 'item_id', 'item_thumbnail','width', 'height']
    no_show_image_columns=['title', 'username','folder', 'item_id', 'width', 'height']
    user = gis.users.get(username)
    username = user.username
    folders = [fld['title'] for fld in user.folders] + [None]
    for folder in folders:
        items = user.items(folder=folder, max_items=1000)
        for item in items:
            thumbnail = item.thumbnail
            if show_image:
                if thumbnail:
                    bn = os.path.basename(thumbnail)
                    image_bytes = item.get_thumbnail()
                    img = Image.open(io.BytesIO(image_bytes))
                    b64_item = base64.b64encode(image_bytes)
                    b64thmb = "data:image/png;base64," + str(b64_item,"utf-8") + "' width='200' height='133"
                    item_thumbnail = """<img src='""" + str(b64thmb) + """' class="itemThumbnail">"""
                    results.append([item.title, username, folder, item.id, item_thumbnail] + list(img.size))
                    img.close()
                    del img
                else:
                    results.append([item.title, username, folder, item.id, "", -999,-999])
            else:
                if thumbnail:
                    image_bytes = item.get_thumbnail()
                    img = Image.open(io.BytesIO(image_bytes))
                    results.append([item.title, username, folder, item.id] + list(img.size))
                    img.close()
                    del img
                else:
                    results.append([item.title, username, folder, item.id,None,None])
    
    if show_image:
        df = pd.DataFrame(results, columns=show_image_columns)
        q = (df.width <= float(min_width)) | (df.height < float(min_height)) | (df.height.isnull()) | (df.width.isnull())
        df = (df[q]
              .copy()
              .reset_index(drop=True))
        return HTML(df.to_html(escape=False))
    else:
        df = pd.DataFrame(results, columns=no_show_image_columns)
        q = (df.width <= float(min_width)) | (df.height < float(min_height)) | (df.height.isnull()) | (df.width.isnull())
        df = (df[q]
              .copy()
             .reset_index(drop=True))
        return df

## Usage

### Example: Retrieve All Thumbnails Under 400x300

In [8]:
username = "geodev0"
df = get_images(username, gis = gis, min_height=300, min_width=400, show_image=False)
df

You do not have permissions to access this resource or perform this operation.


RuntimeError: You do not have permissions to access this resource or perform this operation.
(Error Code: 403)

## Item Thumbnails Back as HTML Report

### Example: Find all Images Under 600x400 Pixels

In [ ]:
df